# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Guillem Barta Gonzàlez<br>
https://github.com/Willy8m/03_Algoritmos/SEMINARIO

Problema:
> 1. Sesiones de doblaje <br>

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. 

Los datos son: 
- Número de actores: 10 
- Número de tomas : 30
- Actores/Tomas : https://bit.ly/36D8IuK
  - 1 indica que el actor participa en la toma
  - 0 en caso contrario

### Información clave

Hay que encontrar que combinación de tomas resulta en el menor número de dias de doblaje.

Restricciones a tener en cuenta:
- Cada actor puede completar cómo máximo, 6 tomas al día.
- Los actores de una misma toma deben asistir el mismo día para grabar dicha toma.

(*) La respuesta es obligatoria

## 0. Importamos librerias y cargamos los datos

In [1]:
import math
import pandas as pd
import numpy as np
from decimal import Decimal

In [2]:
# Load data
df = pd.read_excel('Datos problema doblaje(30 tomas, 10 actores).xlsx', header = 1, index_col = 0)

In [3]:
# Drop columns and rows to keep only the data
data = df[:-2].copy() # drop two last rows
data.drop(columns=["Unnamed: 11", "Total"], inplace=True)  # drop two last columns
data = data.astype(int)
data

,1,2,3,4,5,6,7,8,9,10
Toma,,,,,,,,,,
1,1,1,1,1,1,0,0,0,0,0
2,0,0,1,1,1,0,0,0,0,0
3,0,1,0,0,1,0,1,0,0,0
4,1,1,0,0,0,0,1,1,0,0
5,0,1,0,1,0,0,0,1,0,0
6,1,1,0,1,1,0,0,0,0,0
7,1,1,0,1,1,0,0,0,0,0
8,1,1,0,0,0,1,0,0,0,0
9,1,1,0,1,0,0,0,0,0,0


En la tabla se observan 30 filas, que corresponden a las tomas, y 10 columnas, que corresponden a los actores. Un actor asiste a una toma si hay un 1 en la posición correspondiente.

## 1. ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

El número de posibilidades se rige por la agrupación, sin orden, en grupos de tomas por día. Así, las posibilidades van desde tener todas las tomas el mismo día, a tener todas las tomas separadas en días distintos (caso menos eficiente).

Para calcular el número de agrupaciones diferentes con 30 elementos usamos el número de Bell $B_{n}$ [2], donde "$n$" corresponde al número de elementos. Podemos visualizar el número de Bell con el siguiente dibujo, extraido de wikipedia:

![a](https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Set_partitions_5%3B_circles.svg/220px-Set_partitions_5%3B_circles.svg.png)


De un algoritmo para calcular números de Bell de Rajeev Agrawal [3] se obtiene que existen $8.47E+23$ posibles soluciones:

In [4]:
# code credit: Rajeev Agrawal
# python program to find number of ways of partitioning it.
n = 30
s = [[0 for _ in range(n+1)] for _ in range(n+1)]
for i in range(n+1):
    for j in range(n+1):
        if j > i:
            continue
        elif(i==j):
            s[i][j] = 1
        elif(i==0 or j==0):
            s[i][j]=0
        else:
            s[i][j] = j*s[i-1][j] + s[i-1][j-1]
ans = 0
for i in range(0,n+1):
    ans+=s[n][i]
print("Possibilities: ", '%.2E' % Decimal(ans))

Possibilities:  8.47E+23


## 2. ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?

Teniendo en cuenta las restricciones, el número de posibbles combinaciones de tomas y días dependerá de la ocupación de los actores en cada toma. De módo que el número de posibles agrupaciones depende de los datos específicos de cada problema.

Es posible que exista una estructura de datos más adecuada que nos permita calcular este número.

## 3. ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)

En un principio, elegí una lista de listas binárias, dónde los valores binários representaban la asistencia de un actor a una toma en particular. Es decir, justo la estructura que se nos brinda en el archivo de datos .xlsx

Posteriormente me dí cuenta (después de leer el estudio de Alberto [1]) de que la estructura que se ajusta mejor, es una lista de listas ````sol````, dónde cada lista contiene las tomas que se van a grabar en un determinado día. Con un 1 o un 0 en la posición de la lista correspondiente a cada toma. 

Por ejemplo, consideremos la siguiente solcuión:

In [5]:
# Una toma el primer día, y las otras dos el segundo
#             toma   1 2 3
example_solution = [[1,0,0], # día 1
                    [0,1,1]] # día 2

Además, con esta estructura podemos controlar fácilmente el número de días que se requieren para grabar todas las tomas con:

In [6]:
dias = len(example_solution)
dias

2

Además, esta estructura es conveniente para comprobar que la cantidad de tomas por actor al día no supera el máximo establecido ````max_shots = 6````, con operaciones matriciales ````@```` y ndarrays:

In [7]:
sol = np.array(example_solution)

# Máximo de tomas grabadas al día
max_shots = 1

# Datos de ejemplo:
#            actor 1 2 3
data1 = np.array([[1,1,0], # toma 1
                  [1,1,0], # toma 2
                  [1,0,1]] # toma 3
                )

# Booleana que nos indica si la solución es invalida
invalid_sol = any([any((sol[day] @ data1) > max_shots) for day in range(len(sol))])
invalid_sol

True

Dónde ````data```` es nuestro dataframe de actor por toma

## 4. Según el modelo para el espacio de soluciones ¿Cual es la función objetivo?

### Función objetivo: ````cost````

(Las soluciones son matrices en que cada fila corresponde al día de grabación y las posiciones dentro de esta lista son 1s o 0s si la toma en esa posición se graba ese día.)

In [8]:
def cost(solution, data):
    """Esta función nos devolverá los desplazamientos totales necesarios
    """
    matriz = cantidad_de_tomas_por_actor(solution, data)
    total_cost = np.count_nonzero(matriz)
    return total_cost

In [9]:
def cantidad_de_tomas_por_actor(solution, data):
    """Con esta función relacionamos los datos con nuestra solución
    """
    return solution @ data

In [10]:
# Solución ejemplo:
#      toma   1 2 3
solution1 = [[1,1,1], # día 1
             [0,0,0]] # día 2

# Datos de ejemplo:
#            actor 1 2 3
data1 = np.array([[1,1,0], # toma 1
                  [1,1,0], # toma 2
                  [1,0,1]] # toma 3
                )

print(cantidad_de_tomas_por_actor(solution1, data1))

[[3 2 1]
 [0 0 0]]


In [11]:
cost(solution1, data1)

3

### Función de comprobación de validez: ````is_valid````

Crearemos una función ````is_valid()````, que validará si la solución cumple las restricciones del problema.

In [12]:
def is_valid(sol: np.array, data: np.array, max_shots: int = 6):
    
    # Si falta alguna toma por asignar, devuelve False
    if any(np.sum(sol, axis=0) != 1): return False  

    # Si se usa un actor más de "max_shots" veces en un día, devuelve False
    if any([any((sol[day] @ data) > max_shots) for day in range(len(sol))]): return False

    # En cualquier otro caso la solución es válida y devuelve True
    return True

Por ejemplo si queremos grabar todas las tomas el mismo día, con un máximo de 2 tomas al día por actor, y con los siguientes datos de ejemplo (donde las columnas corresponden a los actores y las filas a las tomas):

In [13]:
# ejemplo: solution1 tiene todas las tomas el mismo día, incumpliendo el máximo de 2 tomas al día por actor
is_valid(solution1, data1, max_shots = 2)

False

## 5. ¿Es un problema de maximización o minimización?

Es un problema de minimización de coste, donde el coste es el número de desplazamientos totales

## 6. Diseña un algoritmo para resolver el problema por fuerza bruta

- **Número máximo de días:** Una toma al día. Por tanto, equivalente al número de tomas.

In [14]:
def max_days(data):
    return len(data)

max_days(data)

30

- **Número mínimo de días:** El número de tomas que graba el actor con más tomas asignadas, dividido por el máximo de tomas que puede grabar ese actor al día.

In [15]:
def min_days(data, max_shots):
    shots = max(np.sum(data, axis = 0))  # Tomas del actor con más tomas
    return math.ceil(shots / max_shots)  # Redondear hacia arriba

min_days(data, max_shots=6)

4

_Dónde ````data```` es nuestro dataframe de actor por toma_

- **Algoritmo por fuerza bruta:** Recorreremos todas las posibles permutaciones de tomas para una cantidad de días N, evaluaremos si son soluciones válidas, y en el caso de que sean válidas, calcularemos su coste. Recorreremos todas las posibilidades hasta encontrar el coste mínimo. Haremos esto desde el mínimo de días posible, hasta el máximo, para no dejarnos ninguna posible organización de días.

In [16]:
def solution_generator(DAYS, SHOTS):

    # Inicializar la solución con todas las tomas en el primer día
    solution = np.zeros((DAYS, SHOTS), dtype=int)
    solution[0] = 1  # Todas las tomas el primer día

    while True:
        yield solution
        if all(solution[-1] == 1):  # Todas las tomas el último día
            return                  # Interrumpimos la generación
        
        # Comprobar qué posición toca cambiar
        last_day_shots = solution[-1]
        c = 0
        for shot in last_day_shots:
            if shot == 1:
                c += 1
            else:
                break
        
        # Aplicar el cambio (rotación)
        for pos in range(0, c + 1):
            shot = solution[:, pos]
            if shot.size > 0:
                last_element = shot[-1]    # Guardar el último elemento
                shot[1:] = shot[:-1]       # Desplazar elementos a la derecha
                shot[0] = last_element     # Colocar el último elemento en la primera posición


In [17]:
def brute_force_solve(data, max_shots=6):

    # Inicializamos variables
    best_cost = data.size  # inicializar con el máximo coste
    min_day = min_days(data, max_shots)
    max_day = max_days(data)
    SHOTS = len(data)

    # MAIN LOOP: Búsqueda por todos los días
    for DAYS in range(min_day, max_day+1):
        for solution in solution_generator(DAYS, SHOTS):
            if is_valid(solution, data, max_shots):
                new_cost = cost(solution, data)
                if new_cost < best_cost:
                    print(f"Optimized solution! Cost: {new_cost}")
                    best_cost = new_cost
                    best_solution = solution.copy()

    # Versón legible, con las tomas que se deben grabar cada día
    readable = {}
    for idx, day in enumerate(best_solution):
        shots = np.where(day == 1)[0]
        k = "Day " + str(idx + 1)
        readable[k] = list(shots)
    
    return {"best solution": readable, "solution cost": best_cost, "array solution": best_solution}

In [18]:
# Datos de ejemplo:
#            actor 1  2  3
data2 = np.array([[1, 1, 0], # toma 1
                  [1, 1, 0], # toma 2
                  [1, 0, 1]] # toma 3
                )
brute_force_solve(data2, max_shots=2)

Optimized solution! Cost: 5
Optimized solution! Cost: 4


{'best solution': {'Day 1': [2], 'Day 2': [0, 1]},
 'solution cost': 4,
 'array solution': array([[0, 0, 1],
        [1, 1, 0]])}

## 7. Calcula la complejidad del algoritmo por fuerza bruta

El algoritmo por fuerza bruta ````brute_force_solve```` tiene una complejidad de $O(n^3)$.

Hay dos bucles for anidados que cuentan cómo $n^2$:

````Python
for DAYS in range(min_day, max_day+1):
    for solution in solution_generator(DAYS, SHOTS):
````

Y dentro de la función ````solution_generator```` hay sólo dos bucles for no anidados entre ellos, que añaden un factor $n$ de complejidad.

## 8. Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

### Solución por búsqueda aleatoria

Vamos a generar un algoritmo de generación de soluciones aleatorias, hasta encontrar una solución que nos dé un número de días suficientemente corto. Aunque vamos a generar soluciones aleatorias, debemos tener en cuenta que existe un número máximo de días (solución menos óptima), al igual que un número mínimo (solución ideal de existencia no asegurada)

In [19]:
def generate_random_solution(data: pd.DataFrame, days_objective: int, max_shots: int = 6):
    
    random_solution = np.zeros((days_objective, len(data)), dtype=int)  # Inicializar el array soluicón, dónde las filas representan los días, y las columnas las tomas
    assigned_days = np.random.randint(0, days_objective, size=len(data))  # Asignar cada toma a un día
    
    for day, toma in enumerate(assigned_days):
        random_solution[toma, day] = 1  # Transformar la asignación de días a formato booleano

    return random_solution

Ahora, vamos a diseñar un algoritmo que recorra un bucle hasta encontrar una solución válida. Además, incluiremos un control para tiempos de iteración demasiado largos, que podremos acotar con el parámetro ````max_iter````.

In [20]:
def generate_valid_solution(ndays: int, data, max_iter=500, max_shots=6):
    for _ in range(max_iter):
        proposed_sol = generate_random_solution(data, days_objective = ndays, max_shots = max_shots)
        if is_valid(proposed_sol, data, max_shots):
            return proposed_sol

    return None

Y, en caso de no encontrar una solución en ese número de iteraciones, buscaremos una solución con un día más.

In [21]:
def random_solve(data: pd.DataFrame, max_shots=6, max_iter=500):

    # Inicializamos variables
    current_cost = data.size  # inicializar con el máximo coste
    min_day = min_days(data, max_shots)
    max_day = math.ceil(len(data) // max_shots)

    # MAIN LOOP: búsqueda de soluciones
    for ndays in range(min_day, max_day + 1):
        for _ in range(max_iter):
            new = generate_valid_solution(ndays, data, max_iter, max_shots)
            if new is not None:
                new_cost = cost(solution=new, data=data)
                if new_cost < current_cost:
                    current_cost = new_cost
                    best_solution = new

    # Versón legible, con las tomas que se deben grabar cada día
    readable = {}
    for idx, day in enumerate(best_solution):
        shots = np.where(day == 1)[0]
        k = "Day " + str(idx + 1)
        readable[k] = list(shots)
    
    return {"best solution": readable, "solution cost": current_cost, "array solution": best_solution}

In [22]:
%%time
random_solve_solution = random_solve(data)

CPU times: total: 7.36 s
Wall time: 7.36 s


In [23]:
random_solve_solution

{'best solution': {'Day 1': [0, 1, 4, 5, 9, 10, 20, 22, 25],
  'Day 2': [6, 8, 16, 21, 26],
  'Day 3': [2, 3, 11, 14, 19, 27, 28],
  'Day 4': [7, 12, 13, 15, 17, 18, 23, 24, 29]},
 'solution cost': 28,
 'array solution': array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
         0, 1, 1, 0, 0, 0, 0, 1]])}

## 9. Calcula la complejidad del algoritmo

La complejidad del algoritmo de búsqueda aleatoria es $O(n^2)$

En el loop de búsqueda de soluciones hay dos bucles anidados, pero el segundo bucle itera sobre una constante definida por ````max_iter````, de modo que esto es una constante y no crece con las dimensiones del problema. En cambio ````ndays```` sí que puede crecer con el tamaño de datos de entrada, que añade un factor $n$ de complejidad.

````Python
for ndays in range(min_day, max_day + 1):
        for _ in range(max_iter):
            new = generate_valid_solution(ndays, data, max_iter)
````

La función ````generate_valid_solution````, también esta limitada por un factor de iteraciones máximas, de módo que no añade orden de complejidad al algoritmo, pero contiene a la función ````generate_random_solution````, que sí que contiene un bucle for que crece con las dimensiones del problema, añadiendo un factor $n$ de complejidad:

````Python
for day, toma in enumerate(assigned_days):
````

## 10. Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Para diseñar un juego de datos aleatorio, pero con sentido, debemos asegurar que todas las tomas tienen al menos un actor presente.
Para controlar la densidad de ocupación de las tomas, es decir, la densidad de población, añadiremos un parámetro:

- ````population_density````: fracción entre 0 y 1 que indicará la fracción de ocupación de tomas respecto al total

In [24]:
def generate_random_dataset(shots: int, actors: int, population_density: float):

    if (population_density <= 0) or (population_density > 1):
        raise ValueError("population_density should be a value in the range (0, 1]")

    # Inicializar el array dataset, dónde las filas representan los tomas, y las columnas los actores
    random_dataset = np.zeros((shots, actors), dtype=int) 

    # Asignar tomas a los actores hasta alcanzar el mínimo de población
    while (np.sum(random_dataset) / random_dataset.size) < population_density:  # Asegurar el mínimo de población
        assigned_actors_to_shot = np.random.randint(0, actors, size=shots)  # Asignar cada toma a un actor
        for shot, actor in enumerate(assigned_actors_to_shot):
            random_dataset[shot, actor] = 1

    return random_dataset

In [25]:
random_dataset = generate_random_dataset(shots=7, actors=7, population_density=.4)

# Comprobamos que no hayan tomas vacías
if all(np.sum(random_dataset, axis=1) > 0):
    print("Éxito! Todas las tomas tienen al menos un actor.")
else:
    print("Error: Hay tomas sin actores.")

random_dataset

Éxito! Todas las tomas tienen al menos un actor.


array([[0, 0, 1, 1, 1, 0, 1],
       [1, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 1]])

## 11. Aplica el algoritmo al juego de datos generado

Aplicaremos los dos algoritmos para comparar su precisión y agilidad. 

Observamos que el algoritmo de búsqueda aleatoria es mucho más rápido que el de fuerza bruta, y consigue una solución con el mismo coste que el de fuerza bruta. Es decir, este algoritmo heurístico es capaz de llegar a una solución òptima mucho más rápido.

### Heurístico

In [29]:
%%time
random_solution = random_solve(random_dataset, max_shots=2, max_iter=500)
random_solution

CPU times: total: 469 ms
Wall time: 496 ms


{'best solution': {'Day 1': [0, 3, 5], 'Day 2': [1, 2, 4, 6]},
 'solution cost': 12,
 'array solution': array([[1, 0, 0, 1, 0, 1, 0],
        [0, 1, 1, 0, 1, 0, 1]])}

### Fuerza bruta

In [30]:
%%time
brute_force_solution = brute_force_solve(random_dataset, max_shots=2)
brute_force_solution

Optimized solution! Cost: 12
CPU times: total: 50.9 s
Wall time: 50.9 s


{'best solution': {'Day 1': [2, 3, 5, 6], 'Day 2': [0, 1, 4]},
 'solution cost': 12,
 'array solution': array([[0, 0, 1, 1, 0, 1, 1],
        [1, 1, 0, 0, 1, 0, 0]])}

## 12. Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

[1] Alberto Caldas Lima, [Aplicación de algoritmos heurísticos para optimizar el coste de doblaje de películas](http://eio.usc.es/pub/mte/descargas/ProyectosFinMaster/Proyecto_759.pdf)

[2] Wikipedia, [Bell Number](https://en.wikipedia.org/wiki/Bell_number)

[3] Rajeev Agrawal, [Bell Numbers (Number of ways to Partition a Set)](https://www.geeksforgeeks.org/bell-numbers-number-of-ways-to-partition-a-set/)

## 13. Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Se debería de plantear un heurístico más eficiente, que hiciera cambios para minimizar el coste, y no de forma aleatoria.